`StructType` and `StructField` With Examples

PySpark `StructType` and `StructField` classes are used to programmatically specify the schema to
the DataFrame and create complex columns like nested struct, array, and map columns.
`StructType` is a collection of `StructField` that defines column name, column data type, boolean 
to specify if the field can be nullable or not metadata. Though PySpark infers the schema from the 
data, sometimes we may need to define our own column names an